# Imports and setup

In [1]:
import os
os.chdir("/home/jptalusan/mta_stationing_problem")

In [2]:
from copy import deepcopy
from src.config import *
from pandas.core.common import SettingWithCopyWarning
from src import data_utils, triplevel_utils
from pyspark.sql import SparkSession

import numpy as np
import datetime as dt
import seaborn as sns
import joblib

import warnings
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
pd.set_option('display.max_columns', None)
import xgboost as xgb

In [3]:
import importlib
importlib.reload(data_utils)

<module 'src.data_utils' from '/media/seconddrive/mta_stationing_problem/src/data_utils.py'>

In [4]:
spark = SparkSession.builder.config('spark.executor.cores', '8').config('spark.executor.memory', '40g')\
        .config("spark.sql.session.timeZone", "UTC").config('spark.driver.memory', '20g').master("local[26]")\
        .appName("wego-daily").config('spark.driver.extraJavaOptions', '-Duser.timezone=UTC').config('spark.executor.extraJavaOptions', '-Duser.timezone=UTC')\
        .config("spark.sql.datetime.java8API.enabled", "true").config("spark.sql.execution.arrow.pyspark.enabled", "true")\
        .getOrCreate()

22/09/08 04:14:14 WARN Utils: Your hostname, scope-vanderbilt resolves to a loopback address: 127.0.1.1; using 10.2.218.69 instead (on interface enp8s0)
22/09/08 04:14:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/08 04:14:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/09/08 04:14:15 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
# load the APC data from a prepared file
processed_triplevel = os.path.join('data', 'processed', 'triplevel_df.parquet')
if not os.path.exists(processed_triplevel):
    filepath = os.path.join(os.getcwd(), "data", "processed", "apc_weather_gtfs.parquet")
    apcdata = spark.read.load(filepath)
    apcdata.createOrReplaceTempView("apc")

    # filter subset
    query = f"""
                SELECT *
                FROM apc
            """
    apcdata=spark.sql(query)
    apcdata = data_utils.remove_nulls_from_apc(apcdata)
    apcdata.createOrReplaceTempView('apcdata')
    apcdata_per_trip = data_utils.get_apc_per_trip_sparkview(spark)
    df = apcdata_per_trip.toPandas()
    
    # Adding extra features
    # Holidays
    fp = os.path.join('data', 'others', 'US Holiday Dates (2004-2021).csv')
    holidays_df = pd.read_csv(fp)
    holidays_df['Date'] = pd.to_datetime(holidays_df['Date'])
    holidays_df['is_holiday'] = True
    df = df.merge(holidays_df[['Date', 'is_holiday']], left_on='transit_date', right_on='Date', how='left')
    df['is_holiday'] = df['is_holiday'].fillna(False)
    df = df.drop(columns=['Date'])
    
    # School breaks
    fp = os.path.join('data', 'others', 'School Breaks (2019-2022).pkl')
    school_break_df = pd.read_pickle(fp)
    school_break_df['is_school_break'] = True
    df = df.merge(school_break_df[['Date', 'is_school_break']], left_on='transit_date', right_on='Date', how='left')
    df['is_school_break'] = df['is_school_break'].fillna(False)
    df = df.drop(columns=['Date'])

    # Traffic
    fp = os.path.join('data', 'traffic', 'triplevel_speed.pickle')
    speed_df = pd.read_pickle(fp)
    speed_df = speed_df[['transit_date', 'trip_id', 'route_id_direction', 'traffic_speed']]
    df = df.merge(speed_df, how='left', 
                  left_on =['transit_date', 'trip_id', 'route_id_direction'], 
                  right_on=['transit_date', 'trip_id', 'route_id_direction'])
    df = df[~df['traffic_speed'].isna()]
    df.to_parquet(processed_triplevel, engine='auto', compression='gzip')
else:
    df = pd.read_parquet(processed_triplevel, engine='auto')
    df = df.dropna()
    # Removing time_window in case a different one will be used
df = df.drop(['time_window', 'load'], axis=1)
df = df.reset_index(drop=True)

## Feature Analysis (used features)
* Datetime: `year`, `month`, `dayofweek`, `hour`, `day`
* GTFS: `scheduled_headway`, `route_direction_name`, `route_id`, `block_abbr`
* Weather: `temperature`, `humidity`, `precipitation_intensity`
* APC data on a stop level is grouped into trips and data is gathered by using the first instance (route_id, route_direction_name) or the average of the numerical values (scheduled headay, weather data)

In [6]:
print(df.shape)
df.head(1)

(430404, 21)


,trip_id,transit_date,arrival_time,year,month,route_id,route_direction_name,block_abbr,dayofweek,hour,temperature,humidity,precipitation_intensity,scheduled_headway,actual_headways,y_reg100,y_reg095,route_id_direction,is_holiday,is_school_break,traffic_speed
0,193715,2020-01-01,2020-01-01 17:24:14,2020,1,14,FROM DOWNTOWN,1400,4,17,49.390999,0.467,0.0,3600.0,3654.976744,9.0,9.0,14_FROM DOWNTOWN,True,True,19.483279


In [13]:
df.groupby('route_id_direction').count().sort_values('trip_id')

,trip_id,transit_date,arrival_time,year,month,route_id,route_direction_name,block_abbr,dayofweek,hour,temperature,humidity,precipitation_intensity,scheduled_headway,actual_headways,y_reg100,y_reg095,is_holiday,is_school_break,traffic_speed
route_id_direction,,,,,,,,,,,,,,,,,,,,
94_FROM NASHVILLE,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
38_TO DOWNTOWN,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
95_FROM NASHVILLE,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
35_TO DOWNTOWN,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
72_GRASSMERE,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22_FROM DOWNTOWN,17937,17937,17937,17937,17937,17937,17937,17937,17937,17937,17937,17937,17937,17937,17937,17937,17937,17937,17937,17937
56_FROM DOWNTOWN,19273,19273,19273,19273,19273,19273,19273,19273,19273,19273,19273,19273,19273,19273,19273,19273,19273,19273,19273,19273
55_FROM DOWNTOWN,19357,19357,19357,19357,19357,19357,19357,19357,19357,19357,19357,19357,19357,19357,19357,19357,19357,19357,19357,19357


## Feature Generation
Generated features, $y_t = f(x_{t-1})$, are always generated using past information.
* `time_window`: Assigning the arrival times into time windows (30 minutes by default).
* `window_of_day`: Just a larger time window (could probably remove)
* `actual_headways`: On a stop level, actual headways are given using the arrival times of the bus to the bus stop. On a trip level, this was averaged over the multiple bus stops across a single trip.
* `congestion_surrogate`: Generated by a model trained on the scheduled and actual headways. (tentatively included, surrogate model is not yet that accurate)
* `route_id_direction`: Combined route_id and route_direction into one feature and then one hot encoded.
* Other categorical values are converted to ordinal integers.

In [ ]:
RANDOM_SEED = 100
WINDOW = 30
PAST_TRIPS = 5
TARGET = 'y_reg100'

In [ ]:
cat_features = ['route_id_direction', 'is_holiday', 'dayofweek', 'is_school_break', 'time_window']
ord_features = ['year', 'month', 'hour', 'day']
num_features = ['temperature', 'humidity', 'precipitation_intensity', 'avg_sched_headway', 'traffic_speed']

In [ ]:
# In the interest of time
tdf = deepcopy(df)

In [ ]:
tdf = triplevel_utils.generate_new_features(df, time_window=WINDOW, past_trips=PAST_TRIPS, target=TARGET)

In [ ]:
# Group by time windows and get the maximum of the aggregate load/class/sched
# Get mean of temperature (mostly going to be equal)
tdf = tdf.groupby(['transit_date', 'route_id_direction', 'time_window']).agg({"year":"first", 
                                                                              "month":"first",
                                                                              "day": "first",
                                                                              "dayofweek":"first", 
                                                                              "hour":"first",
                                                                              "is_holiday": "first",
                                                                              "is_school_break": "first",
                                                                              "temperature":"mean", 
                                                                              "humidity":"mean",
                                                                              "traffic_speed":"mean",
                                                                              "precipitation_intensity": "mean",
                                                                              "scheduled_headway": "max",
                                                                              TARGET: "max"})
tdf = tdf.reset_index(level=[0,1,2])

In [ ]:
display(tdf.head())

In [ ]:
print("ohe_encoder is for the following column order:", cat_features)
rf_df, ix_map, ohe_encoder, percentiles = triplevel_utils.prepare_df_for_training(tdf, cat_features, ord_features, target=TARGET)
percentiles

In [ ]:
drop_cols = ['route_id', 'route_direction_name', 'block_abbr', 'y_reg100', 'y_reg095', 'transit_date', 'is_holiday', 'route_id_direction', 'actual_headways', 'trip_id', 'arrival_time']
drop_cols = [col for col in drop_cols if col in rf_df.columns]
rf_df = rf_df.drop(drop_cols, axis=1)

display(rf_df['y_class'].value_counts())

y = rf_df.pop('y_class')
X = rf_df

In [ ]:
print(X.shape)
X.head(5).style.set_precision(2)

In [ ]:
print(y.unique())
pd.DataFrame(y.head())

In [ ]:
fp = os.path.join('models', 'any_day', 'TL_OHE_encoders.joblib')
joblib.dump(ohe_encoder, fp)
fp = os.path.join('models', 'any_day', 'TL_IX_map.joblib')
joblib.dump(ix_map, fp)
fp = os.path.join('models', 'any_day', 'TL_X_columns.joblib')
joblib.dump(X.columns, fp)

In [ ]:
# Grid search results
fp = os.path.join('models', 'any_day', 'XGBOOST_RANDSEARCHCV_any_day_with_schoolbreak012.joblib')
search_results = joblib.load(fp)
print(search_results.best_params_)

# For bins 012

In [ ]:
# Train on entire dataset

n_estimators  = search_results.best_params_['n_estimators']
max_depth     = search_results.best_params_['max_depth']
learning_rate = search_results.best_params_['learning_rate']
gamma         = search_results.best_params_['gamma']
objective     = 'multi:softmax'

model012 = xgb.XGBClassifier(n_estimators=n_estimators, max_depth=max_depth,
                             learning_rate=learning_rate, use_label_encoder=False, gamma=gamma, num_class=3,
                             objective=objective, eval_metric='mlogloss')
# model012 = xgb.XGBClassifier(use_label_encoder=False, num_class=3,
#                              objective=objective, eval_metric='mlogloss')

model012.fit(X, y, verbose=1)

fp = os.path.join('models', 'any_day', 'XGB_012.joblib')
joblib.dump(model012, fp)

## For bins 234

In [ ]:
rf_df, ix_map, ohe_encoder, percentiles = triplevel_utils.prepare_df_for_training(tdf, cat_features, ord_features, target=TARGET)

# Train 2 separate models for bins 0, 1, 2 and 2, 3, 4
# Adjusting y_class to incorporate Dan's request
# Use Transit's 3 bins as a base. For the highest capacity bin, carve out everything from 55 to 75 as a 4th bin, and 75+ as a 5th bin.

rf_df, percentiles = triplevel_utils.adjust_bins(rf_df, TARGET=TARGET, percentiles=percentiles)
display(rf_df['y_class'].value_counts())
print(percentiles)
drop_cols = ['route_id', 'route_direction_name', 'block_abbr', 'y_reg100', 'y_reg095', 'transit_date', 'is_holiday', 'route_id_direction', 'is_school_break']
drop_cols = [col for col in drop_cols if col in rf_df.columns]
rf_df = rf_df.drop(drop_cols, axis=1)
rf_df = rf_df[rf_df['y_class'] >= 2]
display(rf_df['y_class'].value_counts())

y = rf_df.pop('y_class')
y = y - 2
X = rf_df

In [ ]:
# Grid search results
fp = os.path.join('models', 'any_day', 'XGBOOST_RANDSEARCHCV_any_day_with_schoolbreak234.joblib')
search_results = joblib.load(fp)
print(search_results.best_params_)

In [ ]:
# Train on entire dataset
n_estimators  = search_results.best_params_['n_estimators']
max_depth     = search_results.best_params_['max_depth']
learning_rate = search_results.best_params_['learning_rate']
gamma         = search_results.best_params_['gamma']
objective     = 'multi:softmax'

model234 = xgb.XGBClassifier(n_estimators=n_estimators, max_depth=max_depth,
                            learning_rate=learning_rate, use_label_encoder=False, gamma=gamma, num_class=3,
                            objective=objective, eval_metric='mlogloss')
# model234 = xgb.XGBClassifier(use_label_encoder=False, num_class=3,
#                              objective=objective, eval_metric='mlogloss')

model234.fit(X, y, verbose=1)

fp = os.path.join('models', 'any_day', 'XGB_234.joblib')
joblib.dump(model234, fp)

# Prediction

In [ ]:
fp = os.path.join('models', 'any_day', 'XGB_012_simple.joblib')
model012 = joblib.load(fp)

fp = os.path.join('models', 'any_day', 'XGB_234_simple.joblib')
model234 = joblib.load(fp)

In [ ]:
y_pred = model234.predict(X)
X.iloc[np.where(y_pred == 2)]

In [ ]:
y_pred = model234.predict(X.loc[y[y == 2].index])

In [ ]:
unique, counts = np.unique(y_pred, return_counts=True)
unique, counts

### Actual prediction process
> Can move to separate notebook
0. Mirror streamlit variables
1. Select date to predict
2. Get past data from range that share day of week values
3. Get average of certain values
4. Adjust data for new prediction date
5. Setup, predict and output results

In [ ]:
model012    = joblib.load('models/any_day/XGB_012_simple.joblib')
model234    = joblib.load('models/any_day/XGB_234_simple.joblib')
columns     = joblib.load('models/any_day/TL_X_columns.joblib')
ix_map      = joblib.load('models/any_day/TL_IX_map.joblib')
ohe_encoder = joblib.load('models/any_day/TL_OHE_encoders.joblib')

In [ ]:
import datetime as dt
from src import any_day_prediction_utils as prediction
date_to_predict = dt.date(2022, 4, 1)

In [ ]:
importlib.reload(prediction)

In [ ]:
past_df = prediction.get_past_data(spark, date_to_predict)
input_df = prediction.setup_input_data(date_to_predict, past_df)
display(input_df)
input_df = prediction.prepare_any_day_for_prediction(input_df, columns, ix_map, ohe_encoder)

In [ ]:
len(columns)

In [ ]:
input_df = input_df.reset_index(drop=True)
input_df = input_df[columns]

## Predict first stage 0-1-2
predictions = model012.predict(input_df)

unique, counts = np.unique(predictions, return_counts=True)
print(unique, counts)

input_df['y_pred'] = predictions
## Isolate predictions with bin 2 for 2-3-4
high_bin_df = input_df[input_df['y_pred'] == 2]
display(high_bin_df)
high_bin_df = high_bin_df.drop(['y_pred'], axis=1)
high_bin_index = high_bin_df.index
high_bin_df = high_bin_df[columns]

predictions = model234.predict(high_bin_df)

unique, counts = np.unique(predictions, return_counts=True)
print(unique, counts)

predictions = predictions + 2
input_df.loc[high_bin_index, 'y_pred'] = predictions
ohe_features = ['route_id_direction', 'is_holiday', 'dayofweek', 'is_school_break']
input_df[ohe_features] = ohe_encoder.inverse_transform(input_df.filter(regex='route_id_direction_|is_holiday_|dayofweek_|is_school_'))

results = prediction.generate_results(input_df, TIMEWINDOW=30)

In [ ]:
import seaborn as sns
sns.heatmap(results, annot=False)

In [ ]:
y_pred = model012.predict(input_df)

In [ ]:
unique, counts = np.unique(y_pred, return_counts=True)
unique, counts

In [ ]:
y_pred = model012.predict(X[(X['month_ix'] == 9) & (X['day_ix'] == 10)][columns])
unique, counts = np.unique(y_pred, return_counts=True)
unique, counts

In [ ]:
y_pred = model012.predict(X[columns])
unique, counts = np.unique(y_pred, return_counts=True)
unique, counts

# IGNORE

In [ ]:
import os
import joblib
os.chdir("/media/seconddrive/mta_stationing_problem")
fp = os.path.join('models/any_day/XGBOOST_RANDSEARCHCV_any_day.pkl')
best = joblib.load(fp)
best.best_params_